In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

In [9]:
data = np.load("SpectrumData.npy", allow_pickle=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.T)

In [16]:
class SSDAE(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SSDAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

input_size = 400
hidden_size1 = 128
hidden_size2 = 64
ssdae1 = SSDAE(input_size, hidden_size1)
ssdae2 = SSDAE(hidden_size1, hidden_size2)


In [11]:
#dataloader 构建 + 张量转换
batch_size = 16
data_normalized = np.asarray(data_normalized, dtype=np.float32)
train_loader = DataLoader(TensorDataset(torch.tensor(data_normalized)), batch_size=batch_size, shuffle=True)

In [12]:
loss_function = nn.MSELoss()
optimizer1 = torch.optim.Adam(ssdae1.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(ssdae2.parameters(), lr=0.001)

In [17]:
num_epochs = 50
for epoch in range(num_epochs):
    for data in train_loader:
        x = data[0]
        x = x.view(x.size(0), -1)
        noise_factor = 0.5
        
        # 添加噪声
        noisy_x = x + noise_factor * torch.randn(x.shape)

        # 自编码器 1
        output1 = ssdae1.encoder(noisy_x)
        reconstructed1 = ssdae1.decoder(output1)
        loss1 = loss_function(reconstructed1, x)

        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()

        # 自编码器 2
        output2 = ssdae2.encoder(output1.detach())  # 使用自编码器 1 的编码器输出作为自编码器 2 的输入
        reconstructed2 = ssdae2.decoder(output2)
        loss2 = loss_function(reconstructed2, output1.detach())

        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss1: {loss1.item():.4f}, Loss2: {loss2.item():.4f}')


Epoch [1/50], Loss1: 0.2349, Loss2: 0.2058
Epoch [2/50], Loss1: 0.2259, Loss2: 0.2061
Epoch [3/50], Loss1: 0.2118, Loss2: 0.2093
Epoch [4/50], Loss1: 0.2302, Loss2: 0.2079
Epoch [5/50], Loss1: 0.2346, Loss2: 0.2087
Epoch [6/50], Loss1: 0.2271, Loss2: 0.2078
Epoch [7/50], Loss1: 0.2357, Loss2: 0.2088
Epoch [8/50], Loss1: 0.2384, Loss2: 0.2060
Epoch [9/50], Loss1: 0.2406, Loss2: 0.2091
Epoch [10/50], Loss1: 0.2123, Loss2: 0.2062
Epoch [11/50], Loss1: 0.2300, Loss2: 0.2067
Epoch [12/50], Loss1: 0.2340, Loss2: 0.2086
Epoch [13/50], Loss1: 0.2319, Loss2: 0.2067
Epoch [14/50], Loss1: 0.2275, Loss2: 0.2058
Epoch [15/50], Loss1: 0.2307, Loss2: 0.2115
Epoch [16/50], Loss1: 0.2267, Loss2: 0.2077
Epoch [17/50], Loss1: 0.2310, Loss2: 0.2072
Epoch [18/50], Loss1: 0.2298, Loss2: 0.2115
Epoch [19/50], Loss1: 0.2318, Loss2: 0.2074
Epoch [20/50], Loss1: 0.2282, Loss2: 0.2066
Epoch [21/50], Loss1: 0.2297, Loss2: 0.2104
Epoch [22/50], Loss1: 0.2248, Loss2: 0.2091
Epoch [23/50], Loss1: 0.2278, Loss2: 0.20

In [ ]:
for epoch in range(num_epochs):
    ssdae1.train()
    for batch in train_loader:
        x = batch[0]
        noise_factor = 0.5
        noisy_x = x + noise_factor * torch.randn_like(x)
        noisy_x = torch.clamp(noisy_x, 0., 1.)

        optimizer1.zero_grad()
        outputs = ssdae1(noisy_x)
        loss = loss_function(outputs, x)
        loss.backward()
        optimizer1.step()

    ssdae1.eval()
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}")